In [8]:
import pandas as pd
import re
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.parsing.preprocessing import strip_numeric
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_short

import gensim
all_stopwords = gensim.parsing.preprocessing.STOPWORDS
print(all_stopwords)

frozenset({'ten', 'four', 'bill', 'indeed', 'a', 'someone', 'meanwhile', 'he', 'give', 'computer', 'now', 'already', 'another', 'describe', 'until', 'with', 'within', 'nothing', 'say', 'throughout', 'its', 'hence', 'here', 'wherein', 'two', 'their', 'doing', 'who', 'anywhere', 'do', 'am', 'ourselves', 'does', 'side', 'less', 'against', 'forty', 'while', 'really', 'next', 'whom', 'done', 'was', 'because', 'couldnt', 'some', 'into', 'serious', 'at', 'due', 'put', 'nine', 'etc', 'we', 'thereupon', 'thin', 'third', 'various', 'be', 'get', 'if', 'twenty', 'that', 'eight', 'himself', 'even', 'very', 'hereafter', 'formerly', 'together', 'them', 'mill', 'i', 'didn', 'she', 'detail', 'to', 'something', 'after', 'sometime', 'than', 'see', 'yet', 'by', 'where', 'and', 'few', 'off', 'take', 'via', 'besides', 'why', 'of', 'perhaps', 'much', 'otherwise', 'anyhow', 'thereafter', 'top', 'front', 'those', 'again', 'show', 'several', 'beyond', 'become', 'enough', 'beforehand', 'yours', 'each', 'empty', 

In [3]:
# Function to count total token words in a DataFrame column
def count_words(text_column):
    tokens = 0
    for items in text_column.iteritems():
        n = len(items[1].split())
        tokens += n
    return tokens

In [4]:
tweets = pd.read_csv("tweets_df.csv", index_col=0)
words_earlier = count_words(tweets["Text"])
tweets.head()

,Text,Date,Retweets,Favorites,Mentions,HashTags
0,Is it landlord that has been struggling since ...,2020-08-09 23:55:14+00:00,13,46,NaN,NaN
1,Victorian new cases trending down as impact of...,2020-08-09 23:53:42+00:00,8,29,NaN,#auspol
2,Hello Monday. As Melbourne starts week two in ...,2020-08-09 23:48:01+00:00,7,23,NaN,NaN
3,The ironic part is the same people patting you...,2020-08-09 23:47:48+00:00,5,76,NaN,NaN
4,"In several states, positive coronavirus tests ...",2020-08-09 23:46:39+00:00,13,16,NaN,NaN


In [10]:
# Removing links, hashtags and ampersand attached text from the tweets
tweets_text = pd.DataFrame([re.sub(r"(?:\@|\#|\&|http)\S+", "", item) for item in tweets["Text"]],
                              columns = ["text"])

# Removing non-alphabetic characters
tweets_text["text"] = [strip_non_alphanum(item) for item in tweets_text["text"]]

# Removing numeric characters
tweets_text["text"] = [strip_numeric(item) for item in tweets_text["text"]]

# Removing punctuation characters
tweets_text["text"] = [strip_punctuation(item) for item in tweets_text["text"]]

# Short words removal, minsize 3
tweets_text["text"] = [strip_short(item, minsize=3) for item in tweets_text["text"]]

# All text to lower case
tweets_text["text"] = [item.lower() for item in tweets_text["text"]]

# Removing the stopwords from the tweets and storing in a new dataframe
tweets_text["text"] = [remove_stopwords(item) for item in tweets_text["text"]]
                                   
'''
# Remove everything except text
tweets_text["text"] = [re.sub(r"[^a-zA-Z]+", ' ', item) for item in tweets_text["text"]]
#tweets_text["text"] = [re.sub(r"[^a-zA-Z0-9]+", ' ', item) for item in tweets_text["text"]]

# Remove single character words
tweets_text["text"] = [re.sub(r"\b[a-zA-Z]\b", "", item) for item in tweets_text["text"]]
'''

words_after = count_words(tweets_text["text"])

print("Words before preprocessing: {}".format(words_earlier))
print("Words after preprocessing: {}".format(words_after))
print("Words removed: {}".format(words_earlier-words_after))

tweets_text.head()

Words before preprocessing: 672
Words after preprocessing: 326
Words removed: 346


,text
0,landlord struggling beginning lockdown source ...
1,victorian new cases trending impact stage lock...
2,hello monday melbourne starts week lockdown da...
3,ironic people patting tweet probably agreeing ...
4,states positive coronavirus tests created prob...


In [11]:
tweets_ll = []

for items in tweets_text["text"].iteritems():
    word_list = items[1].split()
    tweets_ll.append(word_list)
    word_list = None
    
tweets_tokenized = pd.DataFrame(tweets_ll)
tweets_tokenized.index += 1
tweets_tokenized

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
1,landlord,struggling,beginning,lockdown,source,income,want,beg,people,better,...,None,None,None,None,None,None,None,None,None,None
2,victorian,new,cases,trending,impact,stage,lockdown,kicks,thanks,staying,...,None,None,None,None,None,None,None,None,None,None
3,hello,monday,melbourne,starts,week,lockdown,day,million,remote,learning,...,titles,young,readers,entertained,zoom,None,None,None,None,None
4,ironic,people,patting,tweet,probably,agreeing,people,protesting,lockdown,end,...,None,None,None,None,None,None,None,None,None,None
5,states,positive,coronavirus,tests,created,problems,reopened,schools,sending,teachers,...,cases,shut,None,None,None,None,None,None,None,None
6,bonding,pets,recent,survey,surveyed,said,pet,helped,lockdown,said,...,cut,working,home,pet,None,None,None,None,None,None
7,happy,birthday,boy,longford,carved,stellar,career,travelling,world,telling,...,lunch,school,pretty,day,forgetful,wait,celebrate,lockdown,None,None
8,bts,lockdown,antis,kpoppies,report,block,check,countries,fanbase,charting,...,properly,platforms,available,enjoy,comeback,army,None,None,None,None
9,world,tiniest,rarest,wild,pigs,swine,fever,lockdown,None,None,...,None,None,None,None,None,None,None,None,None,None
10,peers,claimed,taxpayer,funded,attendance,allowances,working,home,lockdown,rules,...,day,participation,soared,record,levels,None,None,None,None,None


In [12]:
tweets_tokenized.to_csv("tweets_tokenized.csv", index = True, header = False)